## Cargamos Librerias

In [35]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, array_to_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten

## Cargamos nuestra data de entrenamiento

In [57]:
data_train_path = 'Train_set'

# Variaciones en las imágenes.

train_data_generator = ImageDataGenerator(
    rescale=1/255.,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.2
)

# Cargamos y reconficamos las imágenes de entrada para clasificación binaria.

train_data_flow = train_data_generator.flow_from_directory(
    data_train_path,
    target_size = (64, 64),
    batch_size = 32, 
    class_mode = 'binary' 
)

Found 2902 images belonging to 6 classes.


## Cargamos nuestra data para Prueba

In [58]:
data_test_path = 'Test_set'

test_data_generator = ImageDataGenerator(
    rescale=1/255.
)

test_data_flow = test_data_generator.flow_from_directory(
    data_test_path,
    target_size = (64, 64),
    batch_size = 32, 
    class_mode = 'binary'
)

Found 768 images belonging to 6 classes.


## Creación de la Red Neuronal

In [59]:
cnn_v1 = Sequential()
cnn_v1.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(64, 64, 3)))
cnn_v1.add(MaxPool2D(pool_size=(2,2)))
cnn_v1.add(Flatten())
cnn_v1.add(Dense(units=128, activation='relu'))
cnn_v1.add(Dense(units=1, activation='sigmoid'))

c:\Users\Johnatan Ramirez\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [60]:
cnn_v1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Epocas

In [63]:
cnn_v1.fit(train_data_flow,
           steps_per_epoch=20,
           epochs=8,
           validation_steps=100,
           validation_data=test_data_flow)

Epoch 1/8
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 515ms/step - accuracy: 0.1722 - loss: -16975546.0000 - val_accuracy: 0.1823 - val_loss: -21015426.0000
Epoch 2/8
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 405ms/step - accuracy: 0.1499 - loss: -20570502.0000 - val_accuracy: 0.1823 - val_loss: -25217296.0000
Epoch 3/8
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 435ms/step - accuracy: 0.1721 - loss: -24765778.0000 - val_accuracy: 0.1823 - val_loss: -29958210.0000
Epoch 4/8
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 377ms/step - accuracy: 0.1626 - loss: -28940192.0000 - val_accuracy: 0.1823 - val_loss: -35293204.0000
Epoch 5/8
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 232ms/step - accuracy: 0.1452 - loss: -36588572.0000 - val_accuracy: 0.1823 - val_loss: -38545356.0000
Epoch 6/8
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 474ms/step - accuracy: 0.2002 - loss: -37279832.0000 - val_accuracy: 0.1823 - val_loss: -44834484.0000
Epoch 7/8
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 424ms/step - accuracy: 0.1618 - loss: -45031740.0000 - val_accuracy: 0.1823 - val_loss: -52002276.00

# Evaluamos el Modelo en nuestra data de prueba

In [64]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = cnn_v1.evaluate(test_data_flow, steps=100)

print(f'Pérdida en el conjunto de prueba: {test_loss}')
print(f'Precisión en el conjunto de prueba: {test_accuracy}')

100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.1775 - loss: -58743552.0000  
Pérdida en el conjunto de prueba: -59207356.0
Precisión en el conjunto de prueba: 0.1822916716337204


## Hacemos predicciones sobre nuestra data de prueba

In [65]:
import numpy as np

In [66]:
predicciones = cnn_v1.predict(test_data_flow, steps=len(test_data_flow))

# Convertimo las predicciones a 0 y 1
predicciones_binarias = np.where(predicciones > 0.5, 1, 0)

# Mostramos las primeras 20 predicciones
print(predicciones_binarias[:20])

24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 146ms/step
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]


# Metricas Adicionales de Evaluación

In [67]:
from sklearn.metrics import classification_report, confusion_matrix

etiquetas_reales = test_data_flow.classes

# Generamos reporte de clasificación
print(classification_report(etiquetas_reales, predicciones_binarias))

# Imprimimos la matriz de confusión
print(confusion_matrix(etiquetas_reales, predicciones_binarias))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       128
           1       0.18      1.00      0.31       140
           2       0.00      0.00      0.00       124
           3       0.00      0.00      0.00       125
           4       0.00      0.00      0.00       131
           5       0.00      0.00      0.00       120

    accuracy                           0.18       768
   macro avg       0.03      0.17      0.05       768
weighted avg       0.03      0.18      0.06       768

[[  0 128   0   0   0   0]
 [  0 140   0   0   0   0]
 [  0 124   0   0   0   0]
 [  0 125   0   0   0   0]
 [  0 131   0   0   0   0]
 [  0 120   0   0   0   0]]


c:\Users\Johnatan Ramirez\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Johnatan Ramirez\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Johnatan Ramirez\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Validamos una imagen

In [79]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Cargar la imagen, asegurándonos de redimensionarla a 64x64 (igual que en el entrenamiento)
img_path = 'freshOrange (256).jpg'  # Ruta de la imagen que quieres predecir
img = load_img(img_path, target_size=(64, 64))

img_array = img_to_array(img)
img_array = img_array / 255.0

# Agregar una dimensión extra para representar el batch (ya que el modelo espera un batch)
img_array = np.expand_dims(img_array, axis=0)

# Realizar la predicción
prediccion = cnn_v1.predict(img_array)

# Mostrar la predicción
if prediccion > 0.5:
    print("Es la clase 1 (por ejemplo, Manzana)")
else:
    print("Es la clase 0 (por ejemplo, Naranja)")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Es la clase 1 (por ejemplo, Manzana)
